# Download data using datalad from OpenNeuro

**Link to data**: https://openneuro.org/datasets/ds001229/versions/00001

**Citation**: Lewis-Peacock, J. A., Cohen, J. D., & Norman, K. A. (2016). Neural Evidence of the Strategic Choice Between Working Memory and Episodic Memory in Prospective Remembering. Neuropsychologia. https://doi.org/10.1016/j.neuropsychologia.2016.11.006

**Task**: This dataset contains 25 subjects performing a dual-task propspective memory experiment. Participants were shown a series of words while pictures of faces and scenes were presented in the background. Participants performed an ongoing task while monitoring for a picture target (a particular face or a particular scene) to reappear. Each subject performed two versions of the task (WM-bias, EM-bias) in alternating runs of 15 trials. Each subject had 6 functional runs of 301 TRs. Between runs, participants were given a break during which the experimenter checked that the participant was comfortable and alert.


![task](task.png)
**Task diagram**: The dual-task experiment consisted a picture-target detection prospective memory task (“PM”) embedded in an ongoing lexical-decision task (“OG”). Half of the trials were WMbias trials (1-back lexical decisions and a small set of repeating homogeneous pictures) and half were EMbias trials (2-back lexical decisions and a large set of trial-unique heterogeneous pictures). Two-thirds of all trials included both tasks (“PM+OG”), and one-third did not require PM responses (“OG only”)

In [ ]:
%%bash

datalad clone git@github.com:OpenNeuroDatasets/ds001229.git
cd /Users/TienTong/Desktop/utdallas_demo/data/ds001229

for sub_num in 01 02 03 04 05 ; do
    datalad get sub-${sub_num}
done

# View demographic infor of the first 5 participants 

In [1]:
import pandas as pd

demo = pd.read_csv("/Users/TienTong/Desktop/utdallas_demo/data/participants.tsv", sep='\t')
demo.head(5)

,participant_id,sex,age
0,sub-01,M,33
1,sub-02,F,23
2,sub-03,M,23
3,sub-04,M,28
4,sub-05,M,19


# View data structure of the first participant

In [17]:
!tree -L 4 /Users/TienTong/Desktop/utdallas_demo/data/sub-01

/Users/TienTong/Desktop/utdallas_demo/data/sub-01
├── anat
│   ├── sub-01_T1w.json
│   └── sub-01_T1w.nii.gz
└── func
    ├── sub-01_task-em_run-01_bold.json
    ├── sub-01_task-em_run-01_bold.nii.gz
    ├── sub-01_task-em_run-01_events.tsv
    ├── sub-01_task-em_run-02_bold.json
    ├── sub-01_task-em_run-02_bold.nii.gz
    ├── sub-01_task-em_run-02_events.tsv
    ├── sub-01_task-em_run-03_bold.json
    ├── sub-01_task-em_run-03_bold.nii.gz
    ├── sub-01_task-em_run-03_events.tsv
    ├── sub-01_task-wm_run-01_bold.json
    ├── sub-01_task-wm_run-01_bold.nii.gz
    ├── sub-01_task-wm_run-01_events.tsv
    ├── sub-01_task-wm_run-02_bold.json
    ├── sub-01_task-wm_run-02_bold.nii.gz
    ├── sub-01_task-wm_run-02_events.tsv
    ├── sub-01_task-wm_run-03_bold.json
    ├── sub-01_task-wm_run-03_bold.nii.gz
    └── sub-01_task-wm_run-03_events.tsv

2 directories, 20 files


# Run fMRIPrep for first 5 participants

In [ ]:
%%bash

# Installation instructions here: https://www.nipreps.org/apps/docker/

# docker pull nipreps/fmriprep:20.2.3
# pip install --user --upgrade fmriprep-docker

# run
fmriprep_dir=$HOME/Desktop/utdallas_demo/fmriprep
log=$HOME/Desktop/utdallas_demo/log
 
for sub_num in 01 02 03 04 05 ; do
    bids_sub=$HOME/Desktop/utdallas_demo/data/sub-${sub_num}
    for task in em wm ; do
        docker run -ti --rm \
            -v ${bids_sub}:/data:ro \
            -v ${fmriprep_dir}:/out \
            -v ${fmriprep_dir}:/work \
            -v $HOME/Documents/Licenses/freesurfer_license.txt:/opt/freesurfer/license.txt \
            nipreps/fmriprep:20.2.3 \
                /data \
                /out \
                participant \
                -w /work \
                -t ${task} \
                --output-spaces MNI152NLin6Asym:res-2 \
                --nprocs 8 \
                --skip_bids_validation \
                --fs-no-reconall > ${log}/sub-${sub_num}_task-${task}.log
    done
done